In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_nhanes(balanced=True)
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.05)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (2000, 95)
33.65% missing data
Class distribution: (array([1., 2.]), array([1000, 1000]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 1),
                       (46, 9, 5),
                       (96, 11, 1),
                       (96, 13, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 5.38 sec - Loss 0.060790 - ACC 60.35% - ACC Mean 60.35% - AUC 64.99% - AUC Mean 64.99% - Deter 000
Ite 00050 - 0.96 sec - Loss 0.012178 - ACC 61.30% - ACC Mean 61.77% - AUC 65.96% - AUC Mean 66.51% - Deter 009
Ite 00091 - 1.01 sec - Loss 0.007462 - ACC 62.85% - ACC Mean 61.76% - AUC 67.63% - AUC Mean 66.38% - Deter 050
Early stop ite 91, rollback to correction of ite 41, whith acc of 64.15% and auc of 70.1%
OUR METHOD RUN 1/10 - acc: 64.15% +- 0.0% - auc: 70.1% +- 0.0%
Ite 00000 - 1.12 sec - Loss 0.143298 - ACC 60.85% - ACC Mean 60.85% - AUC 66.52% - AUC Mean 66.52% - Deter 000
Ite 00050 - 1.02 sec - Loss 0.054434 - ACC 61.00% - ACC Mean 61.86% - AUC 65.43% - AUC Mean 66.78% - Deter 044
Ite 00056 - 1.25 sec - Loss 0.051197 - ACC 61.10% - ACC Mean 61.78% - AUC 66.17% - AUC Mean 66.60% - Deter 050
Early stop ite 56, rollback to correction of ite 6, whith acc of 63.55% and auc of 69.36%
OUR METHOD RUN 2/10 - acc: 63.55% +- 0.0% - auc: 69.36% +- 0.0%
Ite 00000 - 1.13 sec - Loss

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 62.1% +- 0.0% - auc: 66.19% +- 0.0%
MEAN - acc: 62.15% +- 0.0% - auc: 66.36% +- 0.0%
KNN - acc: 60.9% +- 0.0% - auc: 65.03% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:55<00:00, 89.67it/s]



GAIN RUN 1/10 - acc: 60.7% +- 2.31% - auc: 64.76% +- 3.52%



100%|██████████| 5000/5000 [00:48<00:00, 102.96it/s]



GAIN RUN 2/10 - acc: 62.9% +- 2.71% - auc: 67.6% +- 2.16%



100%|██████████| 5000/5000 [00:55<00:00, 90.31it/s] 


...


GAIN RUN 8/10 - acc: 59.7% +- 3.4% - auc: 64.62% +- 3.22%



100%|██████████| 5000/5000 [01:04<00:00, 77.24it/s]



GAIN RUN 9/10 - acc: 62.4% +- 2.4% - auc: 68.04% +- 2.83%



100%|██████████| 5000/5000 [00:55<00:00, 90.79it/s] 



GAIN RUN 10/10 - acc: 62.45% +- 2.75% - auc: 66.95% +- 2.81%

GAIN GLOBAL - acc: 61.97% +- 1.06% - auc: 66.78% +- 1.23%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 60.85% +- 0.0% - auc: 65.3% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1

MISSFOREST RUN 1/10 - acc: 59.75% +- 2.64% - auc: 64.14% +- 2.01%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8

MISSFOREST RUN 2/10 - acc: 59.95% +- 3.32% - auc: 64.42% +- 3.24%

Iteration: 0
Iteration: 1
Iteration: 2

MISSFOREST RUN 3/10 - acc: 60.75% +- 1.9% - auc: 65.68% +- 2.52%

Iteration: 0
Iteration: 1

MISSFOREST RUN 4/10 - acc: 58.95% +- 3.06% - auc: 63.35% +- 1.91%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 5/10 - acc: 60.7% +- 1.71% - auc: 64.25% +- 2.02%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOREST RUN 6/10 - acc: 60.3% +- 1.78% - auc: 65.24% +- 2.47%

Iteration: 0
Iteration: 1
Iteration: 2

MISSFOREST RUN 7/10 - acc: 59.95% +- 1.86% - auc: 64.85% +- 2.22%

Iteration: 0
Iteration: 1
Iteration: 2

MISSFOREST RUN 8/10 - acc: 63.5% +- 2.59% - auc: 68.01% +- 2.46%

Iteration: 0
Itera

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 61.9% +- 1.93% - auc: 65.69% +- 1.94%
SOFTIMPUTE RUN 2/10 - acc: 61.6% +- 1.39% - auc: 66.91% +- 1.58%
SOFTIMPUTE RUN 3/10 - acc: 59.85% +- 1.31% - auc: 64.03% +- 1.93%
SOFTIMPUTE RUN 4/10 - acc: 62.2% +- 1.51% - auc: 67.34% +- 2.36%
SOFTIMPUTE RUN 5/10 - acc: 61.9% +- 1.53% - auc: 65.77% +- 1.95%
SOFTIMPUTE RUN 6/10 - acc: 60.8% +- 2.13% - auc: 66.28% +- 3.35%
SOFTIMPUTE RUN 7/10 - acc: 61.15% +- 1.93% - auc: 66.38% +- 2.59%
SOFTIMPUTE RUN 8/10 - acc: 59.85% +- 3.87% - auc: 63.76% +- 4.27%
SOFTIMPUTE RUN 9/10 - acc: 62.15% +- 2.34% - auc: 66.44% +- 3.65%
SOFTIMPUTE RUN 10/10 - acc: 59.95% +- 2.41% - auc: 64.52% +- 2.39%

SOFTIMPUTE GLOBAL - acc: 61.14% +- 0.91% - auc: 65.71% +- 1.16%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.7492

SINKHORN RUN 1/10 - acc: 61.95% +- 1.18% - auc: 65.84% +- 1.36%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.7467

SINKHORN RUN 2/10 - acc: 61.0% +- 0.99% - auc: 65.99% +- 1.83%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.7484

SINKHORN RUN 3/10 - acc: 62.9% +- 2.15% - auc: 66.58% +- 1.85%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.8440

SINKHORN RUN 4/10 - acc: 60.0% +- 3.03% - auc: 63.11% +- 2.82%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.7542

SINKHORN RUN 5/10 - acc: 61.6% +- 3.33% - auc: 65.14% +- 4.19%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.7368

SINKHORN RUN 6/10 - acc: 61.7% +- 3.47% - auc: 65.12% +- 4.16%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.7326

SINKHORN RUN 7/10 - acc: 60.15% +- 2.22% - auc: 65.08% +- 2.19%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.7726

SINKHORN RUN 8/10 - acc: 60.7% +- 3.01% - auc: 65.03% 

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 62.05% +- 2.35% - auc: 67.16% +- 2.18%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 59.95% +- 2.17% - auc: 63.99% +- 3.09%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 60.6% +- 1.83% - auc: 64.86% +- 3.34%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 61.05% +- 2.42% - auc: 65.67% +- 2.8%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 60.6% +- 1.46% - auc: 65.3% +- 2.94%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 61.75% +- 2.01% - auc: 66.58% +- 2.14%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 61.8% +- 2.15% - auc: 66.41% +- 1.96%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 59.9% +- 2.44% - auc: 64.54% +- 2.66%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc: 61

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is significantly better than MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC -